<font color='tomato'><font color="#CC3D3D"><p>
# How to Build Neural Networks using Keras

### Import modules

In [7]:
%run import_modules.py  
%matplotlib inline

# For DNN modeling
import tensorflow as tf

# Tensorflow warning off
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
#  tf.logging.set_verbosity(tf.logging.ERROR) <- logging이 버전업그레이드에서 사용 X라함

# from tensorflow import set_random_seed 
# 위에 코드 안됨에 대한 답변: In Tensoflow2 there is no need to perform
# from tensorflow import set_random_seed
# in order to run
# set_random_seed(x)
# (as it was in older version)
# Only have to run
# import tensorflow
# tensorflow.random.set_seed(x)


tf.random.set_seed(2020)
import keras
from keras import backend as K
from keras.layers import * #Input, Dense
from keras.models import * #Model
from keras.optimizers import *
from keras.initializers import *
from keras.regularizers import *
from keras.utils.np_utils import *
from keras.utils.vis_utils import * #model_to_dot

from keras import layers
from keras import models 

### Step 1: Load and process the data

##### Read data

In [ ]:
# pickle파일 저장 및 읽기
# file.to_pickle('test.pickle')
# new_df = pd.read_pickle('test.pickle'); new_df

# import joblib
# joblib.dump((cv, X_train, X_test, y_train, y_test), 'practice1.pkl')

# joblib.dump(
#     {
#         'tfidf' :tfidf,
#         'target_names': train.target_names,
#         'x_train': x_train,
#         'y_train': y_train,
#         'x_test': x_test,
#         'y_test': y_test,
#         },
#     'newsgroup.pkl')


In [56]:
# 앞 단계(Feature Enginnering)에서 저장했던 훈련/평가/적용 데이터를 읽어온다.
X_train, X_test, y_train, y_test, X_dep, ID_dep = pd.read_pickle('case3_train_test.pkl')

# 모델링에 사용되는 최종 학습 및 평가 데이터 확인
X_train.shape, X_test.shape

((2450, 62), (1050, 62))

##### Split data into train & validation set 

In [5]:
# # Train/Validation
# i = int(round(X_train.shape[0] * 0.75,0))
# X_valid, y_valid = X_train[i:], y_train[i:]
# X_train, y_train = X_train[:i], y_train[:i]

### Step 2: Define the model architecture

##### Stack layers from input to output

|모듈|사용 예 및 종류|
|---|:---:|
|Initializer| Dense(16, kernel_initializer='he_normal')|
|Regularizer| Dense(16, kernel_regularizer=l2(0.01))|
|Activation|relu / elu / selu / softmax / tanh / sigmoid / linear|

In [6]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.
SEED = 1              # seed 숫자를 지정
random.seed(SEED)     # Python 고정
np.random.seed(SEED)  # numpy 고정
tf.random.set_seed(SEED) # Tensorflow 고정

In [59]:
model = models.Sequential()
model.add(layers.Dense(16, input_shape=(X_train.shape[1],), activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))

print("#### Sequential Model")
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# validation_data=(X_valid, y_valid)
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),]
hist = model.fit(X_train, y_train, batch_size=64, epochs=40, callbacks=callbacks, 
                 shuffle=False, verbose=1)

loss_and_metric = model.evaluate(X_train, y_train, batch_size=128, verbose=0)
print("train, loss and metric: {}".format(loss_and_metric))
loss_and_metric = model.evaluate(X_test, y_test, batch_size=128, verbose=0)
print("test, loss and metric: {}".format(loss_and_metric))
print(roc_auc_score(y_test, model.predict(X_test)))

#### Sequential Model
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 16)                1008      
_________________________________________________________________
dropout_33 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 8)                 136       
_________________________________________________________________
dropout_34 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 9         
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
2450/2450 [==============================] - 0s 58us/step - loss: 0.7303 - accuracy:

In [38]:
# Input => Dense + Dense + ... => output
# Model(input, output)
 ## 입력층을 먼저 얘기하고, 입력 다음을 케라스에서는 dense라 하는데
    # 이거를 쌓아서 최종 output까지 쌓아줌
# functional x가 무엇이냐
input = Input(shape=(X_train.shape[1],)) # 입력층: X_train의 열의 갯수, 피쳐의 갯수
x = Dense(16, activation='relu')(input) # 다음층부터 하나의 층을 하나의 dense라 함
                                        # 첫번째 은닉층의 뉴런이 16개이다.
                                        # Dense 인자에 kernel_initializer, kernel_regularizer, bias_regularizer 등 잇음
x = Dropout(0.3)(x) # 
x = Dense(8, activation='relu')(x) # 이러한 층의 뉴런의 수를 우리가 지정한다. -
                                    # 함수를 relu를 지정햇는데 sigmoid랑 다른 다른 함수이다.
x = Dropout(0.2)(x)    

output = Dense(1, activation='sigmoid')(x) # 출력층, 하나만 잇으면 되서 뉴런이 1개임(62 -> 16 -> 8 -> 1)
model = Model(input, output) # 모델 셋팅한 것 

# 여기가 굉장히 중요함

##### Summarize & visualize the model

In [39]:
model.summary() # 뉴럴 넷의 상태를 보여줌
Image(model_to_dot(model,show_shapes=True, show_layer_names=False).create(prog='dot', format='png'))
# 웨이트가 1153번이 학습되는 것

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 62)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 16)                1008      
_________________________________________________________________
dropout_13 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 8)                 136       
_________________________________________________________________
dropout_14 (Dropout)         (None, 8)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 9         
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

### Step 3: Choose the optimizer and the cost function
<img align='left' src='http://drive.google.com/uc?export=view&id=1xybqHMvcDDfUYSN3FLQ6Y1708QkiZSJt' style="width: 70%">

*Optimizer의 예:*
- SGD(lr=0.01, momentum=0.9)
- RMSprop(lr=0.001, rho=0.9)
- Adagrad(lr=0.01)
- Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# binary 일때는 시그모이드 
# multi 일때는 softmax를 씀
# 회귀문제일때는 출력노드의 엑티베이션이 없다.

In [ ]:
# 바로 fit 들어가지 않고 케라스는 compile단계가 하나 더 있다.
# 밑에와 같은 것들 설정하는 것

# 옵티마이저: sgd는 경사하강법을 통해서 최적의 파라미터 찾는것, metrics에 
                                #auc 넣으면 속도가 느려서 accuaacy쓰곤 함
    
    # 옵티마이저를 sgd말고 아담같은 다른 더 좋은 옵티마이저를 쓸거냐 말거냐도 중요하다
# loss, optimizer, metrics 설정 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Step 4: Train the model

##### Set learning conditions & fit the model 

In [ ]:
# batch size, epoch, 조기종료조건 등 설정
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),]
            # keras.callbacks.ModelCheckpoint(filepath='best_nn_model.h5', monitor='val_loss', save_best_only=True)]

hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=64, epochs=40, callbacks=callbacks, 
                 shuffle=False, verbose=1)

# 데이터 2000개인가 3000개이를 64개 batch_size로 쪼개서 64개 한꺼번에 계산 및 학습해서 weight값 결정

##### Visualize training history 

In [ ]:
# 학습할때는 계속 내려가고
# validation loss가 내려가다가 올라가는 그런 모양에서 과적합인지 알수잇는것

plt.plot(hist.history["loss"], label="train loss")
plt.plot(hist.history["val_loss"], label="validation loss")
plt.legend()
plt.xlabel('epoch')
plt.title("Loss")
plt.show()

# 뉴럴넷이 고려할게 너무 많아서(하이퍼 파라미터 뿐만 아니라 아키택쳐)까지
# 해야해서 머신러닝보다 더 좋은 모델링을 하기 힘들다
# 맛보기로 일단 해보는 것 

### Step 5: Evaluate the model performance

In [ ]:
# Accuracy

# 사이킷런에서는 score라 하는것을 케라스에서는 evaluteat 이라 함
model.evaluate(X_test, y_test)

In [ ]:
# ROC-AUC
roc_auc_score(y_test, model.predict(X_test))


# 사이킷런은 precit랑 predict_proba가 잇는데 
# 케라스에서는 predict가 predict_proba에 해당하고 사이킷런의 predict을 하려면 따로 설정해야한다.

### Step 6: Predict unseen data

In [ ]:
# Generates output predictions for the input samples
model.predict(X_dep)

In [ ]:
model.predict(X_dep).flatten()

In [ ]:
# Get class labels
np.where(model.predict(X_dep) > 0.5, 1, 0).flatten()

In [ ]:
# Make submissions
pd.DataFrame({'cust_id': ID_dep, 'gender': model.predict(X_dep).flatten()}).to_csv('dnn_submission.csv', index=False)

###  Step 7: Save the model for future use

In [ ]:
# pickle 형식으로 저장
# with open('nn_model.pkl', 'wb') as f:
#    pickle.dump(model, f)

# 추후 저장한 모형 불러올 때: 
# model = pd.read_pickle('nn_model.pkl')

In [ ]:
# h5 형식으로 저장
# model.save('nn_model.h5')

# 추후 저장한 모형 불러올 때: 
# model = load_model('nn_model.h5')

<font color="#CC3D3D"><p>
# End